## WEATHER (2018 - 2024) FOR Saskatchewan ##

In [1]:
import pandas as pd
import os

In [2]:
# Directory containing your CSV files
directory = "C:/Users/chust/OneDrive/Documents/GitHub/Climate-Change-and-its-Economic-Impact-DAB422-24W-004-Group9/Part II/Weather_2018 to 2024_SK"
# TO CHANGE IF RUN BY DIFFERENT USER

# List all CSV files in the directory
files = [file for file in os.listdir(directory) if file.startswith('en_climate_daily_') and file.endswith('.csv')]

# Initialize an empty DataFrame
combined_csv = pd.DataFrame()

# Read and concatenate CSV files
for file in files:
    filepath = os.path.join(directory, file)
    csv = pd.read_csv(filepath)
    combined_csv = pd.concat([combined_csv, csv], ignore_index=True)

# Display the resulting combined DataFrame
print(combined_csv)


        Longitude (x)  Latitude (y) Station Name Climate ID   Date/Time  Year  \
0             -102.57         50.37    BROADVIEW    4010879  2018-01-01  2018   
1             -102.57         50.37    BROADVIEW    4010879  2018-01-02  2018   
2             -102.57         50.37    BROADVIEW    4010879  2018-01-03  2018   
3             -102.57         50.37    BROADVIEW    4010879  2018-01-04  2018   
4             -102.57         50.37    BROADVIEW    4010879  2018-01-05  2018   
...               ...           ...          ...        ...         ...   ...   
127845        -104.00         53.33      NIPAWIN    407N51G  2024-12-27  2024   
127846        -104.00         53.33      NIPAWIN    407N51G  2024-12-28  2024   
127847        -104.00         53.33      NIPAWIN    407N51G  2024-12-29  2024   
127848        -104.00         53.33      NIPAWIN    407N51G  2024-12-30  2024   
127849        -104.00         53.33      NIPAWIN    407N51G  2024-12-31  2024   

        Month  Day Data Qua

In [3]:
# copied the weather data to weather_clean to keep original data
weather_clean = combined_csv.copy()

In [4]:
# to specify which column and values to drop values from
column_to_check = 'Station Name'
values_to_drop = ['COBOURG (AUT)', 'LANGARA ISLAND RCS', 'PORT WELLER (AUT)',
                  'SANDHEADS CS', 'SATURNA ISLAND CS', 'GILLAM', 'KINCARDINE']
    # removing these station names because there are not in the geography dataset; for consistency | checked using Excel
    # GILLAM is for Manitoba
    # KINCARDINE, data is only until 2022; removed for data range consistency
    
# to drop columns
weather_clean = weather_clean[~weather_clean[column_to_check].isin(values_to_drop)]
weather_clean.shape

(127850, 31)

In [5]:
# rename column names for easier transformation/cleaning
weather_clean.rename(columns = {'Longitude (x)': 'Longitude', 'Latitude (y)': 'Latitude'}, inplace=True)

In [6]:
# to load clean geaography data
geography_clean = pd.read_csv('geography_clean.csv')
geography_clean

,GeoUID,Region.Name,provincename,min_longitude,max_longitude,min_latitude,max_latitude
0,1001101,"Division No. 1, Subd. V (SNO)",Newfoundland and Labrador,-53.6508,-53.0045,46.6111,46.9979
1,1001105,Portugal Cove South (T),Newfoundland and Labrador,-53.2618,-53.2498,46.7011,46.7170
2,1001113,Trepassey (T),Newfoundland and Labrador,-53.4166,-53.3158,46.6885,46.7835
3,1001120,St. Shott's (T),Newfoundland and Labrador,-53.5952,-53.5755,46.6289,46.6392
4,1001124,"Division No. 1, Subd. U (SNO)",Newfoundland and Labrador,-53.2651,-52.7789,46.7561,47.2894
...,...,...,...,...,...,...,...
5157,6208068,Umingmaktok (SET),Nunavut,-108.0283,-107.7360,67.6027,67.7568
5158,6208073,Cambridge Bay (HAM),Nunavut,-105.4180,-104.9311,69.0557,69.2229
5159,6208081,Gjoa Haven (HAM),Nunavut,-95.9975,-95.8105,68.6050,68.6769
5160,6208087,Taloyoak (HAM),Nunavut,-93.6245,-93.3805,69.5070,69.5977


In [7]:
# to create a function to check if a point is within the given range
def is_point_in_range(point, min_val, max_val):
    return (point >= min_val) & (point <= max_val)

# to iterate over rows in the weather DataFrame and assign GeoUID based on the conditions
for index, row in weather_clean.iterrows():
    matching_row = geography_clean[
        is_point_in_range(row['Longitude'], geography_clean['min_longitude'], geography_clean['max_longitude']) &
        is_point_in_range(row['Latitude'], geography_clean['min_latitude'], geography_clean['max_latitude'])
    ]
    if not matching_row.empty:
        weather_clean.at[index, 'GeoUID'] = matching_row['GeoUID'].values[0]
        
# to merge the weather and geography_ontario DataFrames based on 'GeoUID'
merged_data = pd.merge(weather_clean, geography_clean, on='GeoUID', how = 'left')

In [8]:
# to check the count of rows of the original data and the clean data
print(weather_clean.shape)
print(merged_data.shape)

(127850, 32)
(127850, 38)


In [9]:
# copying the original merged data to not overwrite the original
monthly_data = merged_data.copy()

In [10]:
# to get mean, and sum of columns
monthly_data_clean = monthly_data.groupby(['Longitude', 
                                      'Latitude',
                                      'Station Name',
                                      'Climate ID',
                                      'Year',
                                      'Month',
                                      'GeoUID',
                                      'Region.Name',
                                      'provincename']).agg({'Max Temp (°C)': 'mean', 
                                                            'Min Temp (°C)': 'mean', 
                                                            'Mean Temp (°C)': 'mean',
                                                            'Total Rain (mm)': 'sum',
                                                            'Total Snow (cm)': 'sum',
                                                            'Total Precip (mm)': 'sum'}).reset_index()
                                      
# to rename the columns to be consistent with the initial data
monthly_data_clean.rename(columns={'Max Temp (°C)' : 'Mean Max Temp (°C)',
                             'Min Temp (°C)' : 'Mean Min Temp (°C)'},
                          inplace = True)

In [11]:
# to create the 'Extr Max Temp (°C)' column
extr_max_temp = monthly_data.groupby(['Longitude', 
                                      'Latitude',
                                      'Station Name',
                                      'Climate ID',
                                      'Year',
                                      'Month',
                                      'GeoUID',
                                      'Region.Name',
                                      'provincename'])['Max Temp (°C)'].max().reset_index()

# to rename the columns to be consistent with the initial data
extr_max_temp.rename(columns={'Max Temp (°C)' : 'Extr Max Temp (°C)'},
                    inplace = True)

# to merge the extr_max_temp column with the dataframe
monthly_data_clean = pd.merge(monthly_data_clean, 
                             extr_max_temp, 
                             how='left', 
                             on=['Longitude', 'Latitude', 
                                 'Station Name', 'Climate ID', 
                                 'Year', 'Month', 'GeoUID', 
                                 'Region.Name', 'provincename'])

In [12]:
# to create the 'Extr Min Temp (°C)' column
extr_min_temp = monthly_data.groupby(['Longitude', 
                                      'Latitude',
                                      'Station Name',
                                      'Climate ID',
                                      'Year',
                                      'Month',
                                      'GeoUID',
                                      'Region.Name',
                                      'provincename'])['Min Temp (°C)'].min().reset_index()
                               
# to rename the columns to be consistent with the initial data
extr_min_temp.rename(columns={'Min Temp (°C)' : 'Extr Min Temp (°C)'},
                    inplace = True)

# to merge the extr_min_temp column with the dataframe
monthly_data_clean = pd.merge(monthly_data_clean, 
                             extr_min_temp, 
                             how='left', 
                             on=['Longitude', 'Latitude', 
                                 'Station Name', 'Climate ID', 
                                 'Year', 'Month', 'GeoUID', 
                                 'Region.Name', 'provincename'])

In [13]:
# convert column to date time format
monthly_data['Date/Time'] = pd.to_datetime(monthly_data['Date/Time'])

# Find the row with the maximum day for each group
last_day = monthly_data.loc[monthly_data.groupby(['Longitude', 'Latitude', 
                                                  'Station Name', 'Climate ID', 
                                                  'Year', 'Month', 'GeoUID', 
                                                  'Region.Name', 'provincename'])['Day'].idxmax()]
  
last_day = last_day[['Longitude', 'Latitude', 'Station Name', 
                      'Climate ID', 'Year', 'Month', 'GeoUID', 
                      'Region.Name', 'provincename', 'Snow on Grnd (cm)']].copy()

# to rename the columns to be consistent with the initial data
last_day.rename(columns={'Snow on Grnd (cm)' : 'Snow Grnd Last Day (cm)'},
                    inplace = True)
                    
## to merge the extr_min_temp column with the dataframe
monthly_data_clean = pd.merge(monthly_data_clean, 
                             last_day, 
                             how='left', 
                             on=['Longitude', 'Latitude', 
                                 'Station Name', 'Climate ID', 
                                 'Year', 'Month', 'GeoUID', 
                                 'Region.Name', 'provincename'])                     

In [14]:
# to replace null values with 0
monthly_data.fillna(0, inplace=True)

In [15]:
# convert to numeric
monthly_data['Spd of Max Gust (km/h)'] = pd.to_numeric(monthly_data['Spd of Max Gust (km/h)'], errors='coerce')

# create spd max gust by getting the max from the same column
max_speed_index = monthly_data.groupby(['Longitude', 'Latitude', 'Station Name', 'Climate ID', 'Year', 'Month', 'GeoUID', 'Region.Name', 'provincename'])['Spd of Max Gust (km/h)'].idxmax()
spd_max_gust = monthly_data.loc[max_speed_index, ['Longitude', 'Latitude', 'Station Name', 'Climate ID', 'Year', 'Month', 'GeoUID', 'Region.Name', 'provincename', 'Spd of Max Gust (km/h)', 'Dir of Max Gust (10s deg)']]

                   
## to merge the extr_min_temp column with the dataframe
monthly_data_clean = pd.merge(monthly_data_clean, 
                             spd_max_gust, 
                             how='left', 
                             on=['Longitude', 'Latitude', 
                                 'Station Name', 'Climate ID', 
                                 'Year', 'Month', 'GeoUID', 
                                 'Region.Name', 'provincename'])

monthly_data_clean.tail().T

,4195,4196,4197,4198,4199
Longitude,-102.12,-102.12,-102.12,-102.12,-102.12
Latitude,49.32,49.32,49.32,49.32,49.32
Station Name,OXBOW,OXBOW,OXBOW,OXBOW,OXBOW
Climate ID,4015800,4015800,4015800,4015800,4015800
Year,2024,2024,2024,2024,2024
Month,8,9,10,11,12
GeoUID,4701036.0,4701036.0,4701036.0,4701036.0,4701036.0
Region.Name,Moose Creek No. 33 (RM),Moose Creek No. 33 (RM),Moose Creek No. 33 (RM),Moose Creek No. 33 (RM),Moose Creek No. 33 (RM)
provincename,Saskatchewan,Saskatchewan,Saskatchewan,Saskatchewan,Saskatchewan
Mean Max Temp (°C),NaN,NaN,NaN,NaN,NaN


In [16]:
# to check that it's for 1 province only
monthly_data_clean['provincename'].unique()

array(['Saskatchewan'], dtype=object)

In [17]:
# to check the count of rows of the original data and the clean data
print(weather_clean.shape)
print(monthly_data_clean.shape)

(127850, 32)
(4200, 20)


In [18]:
# to convert to string
monthly_data_clean['GeoUID'] = monthly_data_clean['GeoUID'].astype('int').astype('object')
# to convert to datetime format
monthly_data_clean['Date/Time'] = pd.to_datetime(monthly_data_clean[['Year', 'Month']].assign(DAY=1))
# to check dataset
monthly_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Longitude                  4200 non-null   float64       
 1   Latitude                   4200 non-null   float64       
 2   Station Name               4200 non-null   object        
 3   Climate ID                 4200 non-null   object        
 4   Year                       4200 non-null   int64         
 5   Month                      4200 non-null   int64         
 6   GeoUID                     4200 non-null   object        
 7   Region.Name                4200 non-null   object        
 8   provincename               4200 non-null   object        
 9   Mean Max Temp (°C)         3469 non-null   float64       
 10  Mean Min Temp (°C)         3470 non-null   float64       
 11  Mean Temp (°C)             3469 non-null   float64       
 12  Total 

In [19]:
# to arrange columns
monthly_data_clean = monthly_data_clean[['Longitude', 'Latitude', 'Station Name', 
                                         'Climate ID', 'Date/Time', 'Year', 'Month', 
                                         'Mean Max Temp (°C)', 'Mean Min Temp (°C)',
                                         'Mean Temp (°C)','Extr Max Temp (°C)',
                                         'Extr Min Temp (°C)', 'Total Rain (mm)',
                                         'Total Snow (cm)', 'Total Precip (mm)',
                                         'Snow Grnd Last Day (cm)', 'Dir of Max Gust (10s deg)',
                                         'Spd of Max Gust (km/h)','GeoUID', 'Region.Name', 'provincename']]

In [20]:
# to check date range
print(monthly_data_clean['Date/Time'].min())
print(monthly_data_clean['Date/Time'].max()) # to remove data after Jan 2024 since there are incomplete

2018-01-01 00:00:00
2024-12-01 00:00:00


In [21]:
monthly_data_clean = monthly_data_clean[monthly_data_clean['Date/Time'] <= '2024-01-01']

In [22]:
# to check date range; confirm that only unti; Jan 2024
print(monthly_data_clean['Date/Time'].min())
print(monthly_data_clean['Date/Time'].max())

2018-01-01 00:00:00
2024-01-01 00:00:00


In [23]:
# to output weather data 2018 to 2024 into csv format

csv_path = 'weather_geography_merged_2018_to_2024_SK.csv'
monthly_data_clean.to_csv(csv_path, 
                   index = False)

In [24]:
# to check converted data if consistent with the expected values (canada site)
weather_2024 = monthly_data_clean[monthly_data_clean['Date/Time'] >= '2024-01-01']
weather_2024.head().T

,72,156,240,324,408
Longitude,-109.51,-109.5,-109.47,-109.2,-108.99
Latitude,49.64,50.91,49.9,53.45,49.44
Station Name,CYPRESS HILLS PARK,LEADER AIRPORT,MAPLE CREEK,BUTTE ST PIERRE,EASTEND CYPRESS (AUT)
Climate ID,4031999,402DAF0,4024919,4041000,4032322
Date/Time,2024-01-01 00:00:00,2024-01-01 00:00:00,2024-01-01 00:00:00,2024-01-01 00:00:00,2024-01-01 00:00:00
Year,2024,2024,2024,2024,2024
Month,1,1,1,1,1
Mean Max Temp (°C),-5.609677,-9.116129,-3.933333,-11.548387,-8.122581
Mean Min Temp (°C),-15.416129,-20.390323,-17.323333,-20.66129,-16.496774
Mean Temp (°C),-10.512903,-14.73871,-10.62,-16.125806,-12.309677


In [25]:
# to extract the station name and climate ID for each provincename; for further weather data extraction

downloaded_SK = monthly_data_clean[['provincename', 'Station Name', 'Climate ID']].drop_duplicates()
downloaded_SK_df = pd.DataFrame(downloaded_SK, columns = ['provincename', 'Station Name', 'Climate ID'])

csv_path = 'downloaded_SK.csv'
downloaded_SK_df.to_csv(csv_path,
                            index = False)